In [ ]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

### Globale Variablen

Diese Variablen werden im Verlauf des Notebooks über den Scope einzelner Funktionen hinweg benötigt. Um Parameter- und Rückgabelisten kompakt zu halten, werden sie global definiert. Die Erklärung der Verwendung befindet sich im weiteren Verlauf des Notebooks.
* `g_used_boards`: Die Menge der bereits in einem $S_n$ verwendeten Situationen.
* `g_user_wants_pawn`: Flag, ob der Nutzer einen Bauern konfiguriert hat.

In [ ]:
g_used_boards = set()
g_user_wants_pawn = chess.Piece.from_symbol("P") in g_piece_list

# Berechnung der Endspieldatenbank

Wie in Notebook ``01_chess_introduction`` bereits erklärt, ist ein Schachspiel gewonnen, wenn die gegnerische Figur mattgesetzt wurde. 

Bei einer geringen Anzahl an Figuren (in einer Menge $P$) im Spielzustand lassen sich alle möglichen Positionen berechnen.
Ausgehend von allen Positionen mit einem Schachmatt, können Zugfolgen bestimmt werden, welche zu einer Spielsituation $n$ Halbzüge vom Sieg entfernt führen.

Im weiteren Verlauf werden folgende Definitionen verwendet:
* $\mathtt{board.pieces}$: Liste der Figuren, welche in einem Zustand vorhanden sind.
* $\mathtt{valid\_boards}$: Alle Zustände des Schachspiels, die gegen keine Regeln verstoßen.  
  Relevante Regeln sind:
  * Zwei Könige auf dem Spielfeld.
  * Die Könige nicht auf benachbarten Spielfeldern.
  * Keine Bauern auf der 1. und 8. Zeile.
* $\mathtt{won\_boards}$: Alle Zustände des Schachspiels, in denen ein Spieler gewonnen hat.
* $\mathtt{previous\_states(b)}$: Alle Zustände, aus denen durch Ausführen eines einzelnen Zugs der Zustand $b$ erreicht werden kann.

Seien alle möglichen (validen) Kombinationen von Positionen der Figuren in $P$ die Menge $S$.
Für $S$ gilt:  
* $board \in S \implies \forall p \in P : p \in \mathtt{board.pieces}$
* $board \in S \implies board \in \mathtt{valid\_boards}$

In einem regulären Schachspiel müssen auch Stellungen mit geschlagenen Figuren betrachtet werden.  
Demnach wäre die Anforderung $board \in S \implies \forall p \in P : p \in \mathtt{board.pieces}$ nicht immer gegeben.
Aus der Aufgabenstellung (Aufgeführt in `01_introduction.ipynb`) folgt jedoch, dass in den Spielsituationen keine Figur geschlagen und das Spiel noch gewonnen werden kann. Aus diesem Grund werden Szenarien, in welchen Figuren geschlagen werden können, nicht unterstützt.

Aus der Menge $S$ mit allen möglichen Stellungen lassen sich die Zustände auswählen, welche $n$ Halbzüge vom Sieg entfernt sind. 
Diese Zustände werden als die Menge $S_n$ bezeichnet. 
Ist das Spiel gewonnen, verbleiben 0 Züge bis zum Sieg. Für alle diese Zustände, in denen ein Spieler mattgesetzt ist, gilt:  
  
$$board \in S \land board \in won\_boards \implies board \in S_0$$  

Aus dieser Definition können induktiv die verbleibenden $S_n$ hergeleitet werden:  
$$board \in S_{n+1} \iff board \in S \land \forall b \in \mathtt{next\_states(board)}: b \in S_{m} \land m < n+1$$

Durch die vielen unterschiedlichen Möglichkeiten, ein Spiel zu beenden, kann eine Situation in mehreren $S_n$ Mengen sein.
Es gibt jedoch immer einen optimalen Weg, mit der geringsten nötigen Anzahl an Zügen $n$.
Da die Mengen aufsteigend ($n \rightarrow n+1$) bestimmt werden, ist die erste Zuordnung in ein $S_n$ die Optimale.
Um Dopplungen zu vermeiden, können Situationen, welche einem $S_n$ zugeordnet wurden, aus $S$ entfernt werden.

Aus der Aufgabenstellung folgt, dass die Tabellen nur für Weiß als Gewinner des Spiels bestimmt werden.
Folglich wird die KI bei der Auswertung die Züge für Weiß vorgeben.
Die Züge von Schwarz bleiben unvorhersehbar.
Die vorher genannten Definitionen können daher wie folgt angepasst werden:
 * $board \in S \land board \in won\_boards \implies board \in S_0 \land board \notin S$  
 * $board \in S \land \exists b \in S_n: board \in \mathtt{previous\_states(b)} \implies board \in S_{n+1} \land board \notin S$
 * $board \in S \land \forall b \in \mathtt{next\_states(board)}: b \in S_{m} \land m < 2n \implies board \in S_{2n} \land board \notin S$

Um die Effizienz der Rechnung zu steigern, berechnet dieses Notebook nicht alle Stellungen $S$ und entfernt daraus die Stellungen für ein $S_n$ wie zuvor beschrieben.
Stattdessen werden alle bekannten Stellungen in `g_used_boards` gespeichert.
Dopplungen werden also nicht vermieden, indem Stellungen aus einer großen Liste entfernt werden, sondern eine Liste der entfernten Stellungen geführt und neue Stellungen mit dieser abgeglichen werden.

Bevor die eigentliche Berechnung der Endspieltabellen durchgeführt werden kann, müssen einige Hilfsfunktionen definiert werden.

## Allgemeines Konzept der konkreten Berechnung
Die zuvor definierte Bestimmung der $S_n$ findet in diesem Notebook in mehreren Schritten statt.
Zuerst werden die Figuren iterativ auf einem Schachbrett positioniert, um alle möglichen Kombinationen zu erhalten.
Die so entstehenden Situationen, in welchen Schwarz schachmatt ist, werden als $S_0$ gespeichert.
Anschließend werden iterativ für jede Situation in einem $S_n$ alle noch unbekannten (nicht in der Menge `g_used_boards`) Situationen bestimmt, welche einen Halbzug entfernt sind.
Nach jeder Berechnung eines $S_n$ wird das Ergebnis in einer temporären Datei zwischengespeichert.


## Ergebnisverwaltung
Für die Verwendung als Endspiel-KI, müssen die $S_n$ Mengen in einer Form gespeichert werden, in welcher eine spezifische Situation gefunden werden kann, und anschließend ein Pfad zu $S_0$ bestimmt werden.

Im Verlauf der Berechnung und Verwendung müssen mehrfach Vergleiche einer Situation mit einer großen Menge anderer Situationen durchgeführt werden.
Da der Abgleich mit Listen in Python ineffizient ist, werden Mengen verwendet.

Mengen werden in Python als Hash-Tabellen umgesetzt. Dies ist im [Quellcode](https://github.com/python/cpython/blob/main/Objects/setobject.c) zu erkennen. Daraus folgt eine Zeitkomplexität bei der Überprüfung, ob sie ein bestimmtes Element enthalten, von $\mathcal{O}(1)$. Listen hingegen haben für den gleichen Abgleich eine Komplexität von $\mathcal{O}(n)$. Die Zeitkomplexitäten können im [Python-Wiki](https://wiki.python.org/moin/TimeComplexity) nachgelesen werden.

Die `chess.Board` Objekte der `chess` Library sind nicht "Hashbar".
Weiter benötigt einer Ausgabe von ``sys.getsizeof`` zufolge ein Board-Objekt 48 Byte Speicher. Das ist bereits doppelt so viel, wie die 24 Byte eines Standard-Integers. ``sys.getsizeof`` bestimmt jedoch nur die Größe des eigentlichen Objektes. Verwaltet ein Objekt Referenzen auf andere Objekte, ist die tatsächliche Größe höher.

Im Sinne von geringer Laufzeit und Speicherauslastung wird daher für die Verwendung in Python-Mengen mit einer Integer-Repräsentation der Stellungen gearbeitet. 

Diese Repräsentationen werden bestimmt, indem die Positionen der Figuren, welche in der globalen Variable `g_piece_list` aufgeführt werden, binär zusammengefügt werden. Weiter wird ein Bit gesetzt, um darzustellen, welcher Spieler am Zug ist.
So wird aus der folgenden Stellung:
```
. . . . k . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . Q . .
. . . . K . . .
```
Der Integer: `460155`.

Eine detaillierte Erklärung der (De-)Kodierung und Hilfsfunktionen befindet sich im Notebook `12_integer_management.ipynb`.

In [ ]:
%run 12_integer_management.ipynb

## Spiegelungen von Situationen
In diesem Notebook werden Spiegelungen der Stellungen verwendet.

Durch die zuvor in `01_introduction.ipynb` erklärten Bewegungsmuster der Figuren sind Schachbretter in vielen Fällen symmetrisch.

Eine Position mit dem Turm in "a8", der Dame in "g6" und dem gegnerischen König in "h8" ist genauso verloren wie 
dieselbe Position nur mit dem Turm in "a1", der Dame in "g3" und dem König in "h1".
![Mirroring_Example_1](Images/Mirroring_Example_1.png)
![Mirroring_Example_2](Images/Mirroring_Example_2.png)
Dies wäre eine Spiegelung entlang der horizontalen zwischen den Zeilen 4 und 5.
Weiter sind auch Spiegelungen entlang der vertikalen (Zwischen Reihe e und f), den Diagonalen und Rotationen 
(jeweils um 90°, 180° und 270°) möglich.

Durch das simple Spiegeln der Spielsituationen können aus einer validen Spielsituation bis zu sieben weitere ohne
großen Rechenaufwand bestimmt werden. Aus diesem Grund werden in diesem Dokument bei jeder Berechnung neuer Stellungen diese gespiegelt und die Spiegelungen ebenfalls abgespeichert. 

Da Bauern sich nur in eine Richtung bewegen können, gelten für diese andere Regeln beim Spiegeln. 
Insbesondere kann für diese Spielsituationen nur eine Spiegelung durchgeführt werden.
Um die Komplexität der Anwendung nicht weiter zu steigern, werden Spielsituationen mit Bauern nicht gespiegelt.

Die zum Spiegeln verwendeten Funktionen und deren Erklärung befinden sich im Notebook `13_mirroring.ipynb`.

In [ ]:
%run 13_mirroring.ipynb

## Hilfsfunktionen für die Ergebnisverwaltung

Zuvor wurden bereits das Konzept der globalen Menge `g_used_boards`, der Integer-Repräsentationen und der Spiegelung von Situationen beschrieben. Um den Umgang mit diesen in der Berechnung zu vereinfachen, sollen zwei Funktionen definiert werden.

Die Funktion `add_original_and_mirrors_to_used(board_int)` fügt die Integer-Repräsentation einer Stellung, sowie alle Spiegelungen in die globale Variable `g_used_boards` ein.
Hierfür werden die zuvor beschrieben Hilfsfunktionen zum Spiegeln verwendet.

In [ ]:
def add_original_and_mirrors_to_used(board_int):
    global g_used_boards
    g_used_boards.add(board_int)
    if not g_user_wants_pawn:
        for reflection in mirror_all_directions(board_int, g_piece_list):
            g_used_boards.add(reflection)

Die Funktion `add_all_to_used(boards_int)` fügt die Originale und Spiegelungen aller Stellungen in der Menge `boards_int` zur globalen Variable `g_used_boards` hinzu.
Nachdem die eindeutigen Spielsituationen für eine $S_n$ Menge berechnet wurden, kann der zuvor beschriebene Prozess mit der Funktion eingeleitet werden. 
Die Funktion überprüft, ob Spiegelungen durchgeführt werden sollen.
Wenn Spiegelungen durchgeführt werden, wird hierzu die Funktion `add_original_and_mirrors_to_used(boards_int)` verwendet.

In [ ]:
def add_all_to_used(boards_int):
    global g_used_boards
    
    if not g_user_wants_pawn:
        for board_int in boards_int:
            add_original_and_mirrors_to_used(board_int)
    else:
        for board_int in boards_int:
            g_used_boards.add(board_int)

## Berechnung der $S_n$ Mengen
Mit den zuvor beschriebenen Konzepten können die Endspieltabellen berechnet werden.

Der Prozess zur Bestimmung aller Mengen lautet wie folgt:
1. Bestimmen von $S_0$.
2. Solange weitere Mengen gefunden werden, werden alle Spielsituationen, die in einem Halbzug in $S_n$ enden, bestimmt.
3. Die bestimmten Situationen werden zu $S_n+1$ hinzufügt.

### Bestimmung von $S_0$

$S_0$ ist die Menge der Situationen, welche keinen Zug von einem Sieg entfernt sind. Sie sind also bereits gewonnen.
Diese erste Menge wird bestimmt, indem alle möglichen Kombinationen der eingestellten Figuren erstellt und auf ein Schachmatt überprüft werden.
Die Kombinationen werden mit der Funktion `fill_boards_with_piece(boards_int_set, piece, cur_piece_list)` erstellt, welche später in ``calculate_s0()`` eingesetzt wird.

Die Funktion ``fill_boards_with_piece(boards_int_set, piece, cur_piece_list)`` erstellt aus einer Menge an Stellungen (in Integer-Representation) `int_boards_set` und einer Figur `piece` eine Menge von Stellungen, welche die Positionen in `int_board_sets` an jeder freien Stelle um `piece` ergänzen.
Jedes Mal, wenn eine Figur platziert wird, wird eine Kopie des Board-Objektes erstellt.

Wenn der zweite König platziert wird, wird die Stellung zusätzlich auf Validität überprüft.
Wenn alle Figuren platziert wurden, werden nur Boards, in denen Schwarz Schachmatt ist, zurückgegeben.

Die Funktion benötigt als weiteren Parameter die Piece List (`cur_piece_list`), welche die Figuren enthält, die in `boards_int_set` verwendet werden.
Diese Liste wird für das Erstellen von Board-Objekten aus Integern und dem Erstellen von Integern verwendet.

In [ ]:
def fill_boards_with_piece(boards_int_set, piece, cur_piece_list):
    finished_boards = set()
    all_squares = set(range(64))
    piece_count = len(g_piece_list)
    for int_board in boards_int_set:
        board_o = to_board(int_board, cur_piece_list)
        used_squares = set(board_o.piece_map().keys())
        for square in all_squares:
            if square not in used_squares:
                tmp_piece_list = cur_piece_list.copy()
                board_object = board_o.copy()
                board_object.set_piece_at(square, piece)
                tmp_piece_list.append(piece)
                board_int = to_integer(board_object, tmp_piece_list)
                if len(used_squares) > 1 and not board_object.is_valid():
                    # Don't process invalid boards further than the second king
                    continue

                if board_object.is_checkmate():
                    if board_int not in g_used_boards:
                        finished_boards.add(board_int)
                        add_original_and_mirrors_to_used(board_int)
                        continue

                if len(used_squares) + 1 < piece_count: 
                    #Board is valid, but needs more pieces
                    finished_boards.add(board_int)
    return finished_boards

Die Funktion `calculate_s0()` berechnet die Menge aller Stellungen, welche von Weiß gewonnen wurden.
Hierfür wird ein leeres Schachbrett erstellt (`empty_board`) und jede Figur in der `g_piece_list` mit der `fill_boards_with_piece(boards_int_set, piece, cur_piece_list)` Funktion in allen Konstellationen auf Board-Objekten verteilt.
Die Funktion liefert eine Menge von Integer Repräsentationen der Stellungen in $S_0$. 

In [ ]:
def calculate_s0():
    global g_piece_list

    s0 = set()
    empty_board = chess.Board(None)
    # Turn has to be black while checkmate
    empty_board.turn = chess.BLACK

    # Create tmp_piece_list for conversion between board_object and integer
    tmp_piece_list = []
    s0.add(to_integer(empty_board, tmp_piece_list))

    for piece in g_piece_list:
        s0 = fill_boards_with_piece(s0, piece, tmp_piece_list)
        tmp_piece_list.append(piece)
        
    return s0

### Bestimmung aller weiteren $S_n$ Mengen

Die weiteren $S_n$ Mengen werden iterativ erzeugt. Jede Iteration erhält hierbei eine zuvor bestimmte Menge $S_n$ und bestimmt die folgende Menge $S_{n+1}$.
Die Mengen (welche je einen Halbzug entfernt sind) werden bestimmt, indem all diese Halbzüge durchgeführt werden, welche für den jeweils anderen Spieler möglich sind.
Eine Ausnahme zu dieser Methode sind alle Situationen, welche Bauern beinhalten. Bauern können sich nur in eine Richtung bewegen und werden daher nicht über Züge, sondern Versetzen der Figuren bewegt.
  

#### Berechnung der Spielzüge für Stellungen ohne Bauern

Die Funktion `moves_without_pawns(chess_board)` führt für ein Objekt `chess_board` alle möglichen Züge der Figuren aus.

Da die Rechnungen rückwärts (von $S_n$ zu $S_{n+1}$) durchgeführt werden, muss der Spieler am Zug vor dem Hinzufügen zur Ergebnisliste `reached_boards` gewechselt werden.

Der erste Wechsel findet bereits in der `previous_states(sn)` Funktion statt.
Nach Durchführen eines Halbzuges, wird automatisch der Spieler gewechselt, jedoch soll die Funktion Situationen mit dem Spieler ursprünglich am Zug zurückgeben. Aus diesem Grund wird der Zug zweimal getauscht.

Die Funktion gibt alle entstehenden Situationen als `reached_boards` zurück.

In [ ]:
def moves_without_pawns(chess_board):
    reached_boards = []
    for pLMove in chess_board.pseudo_legal_moves:
        # Backwards-execution of Move
        chess_board.push(pLMove)
        chess_board.turn = chess_board.turn ^ True
        reached_boards.append(chess_board.copy())
        chess_board.turn = chess_board.turn ^ True
        chess_board.pop()

    return reached_boards

#### Berechnung der Spielzüge für Stellungen mit Bauern

Die Funktion `check_for_pawn(chess_board)` überprüft, ob sich Bauern auf einer Stellung `chess_board` befinden.

In [ ]:
def check_for_pawn(chess_board):
    return len(chess_board.pieces(chess.PAWN, chess.WHITE)) > 0

Die Funktion `find_pawn(chess_board)` gibt die Nummer des Feldes zurück, in welchem sich ein Bauer auf der Stellung in `chess_board` befindet.

In [ ]:
def find_pawn(chess_board):
    return chess_board.pieces(chess.PAWN, chess.WHITE).pop()

Die Funktion `move_pawn(chess_board)` setzt einen Bauern in der Situation `chess_board` auf ein Feld in einer niedrigeren Reihe.

Befindet sich der Bauer in der vierten Reihe, wird er sowohl in die dritte als auch zweite Reihe gesetzt.
In den Reihen 3,5,6 und 7 wird er jeweils eine Reihe zurückgesetzt.

Die Rückgabe `res` enthält demnach 0 - 2 Board Objekte.

In [ ]:
def move_pawn(chess_board):
    res = []
    position = find_pawn(chess_board)
    if position in range(24,31):
        tmp_board = chess_board.copy()
        tmp_board.remove_piece_at(position)
        tmp_board.set_piece_at(position - 16, chess.Piece.from_symbol('P'))
        res.append(tmp_board)
    
    if position in range(16,63): 
        chess_board.remove_piece_at(position)
        chess_board.set_piece_at(position - 8, chess.Piece.from_symbol('P'))
        res.append(chess_board)
        
    return res

Die Funktion `check_top_row_for_queen(chess_board)` überprüft für eine Situation `chess_board`, ob sich in der 8. Reihe eine Dame befindet.
Die Rückgabe erfolgt als boolescher Wert.

In [ ]:
def check_top_row_for_queen(chess_board):
    square_with_queen = chess_board.pieces(chess.QUEEN, chess.WHITE).pop()
    return square_with_queen in range(56,64)

Die Funktion `replace_queen(chess_board)` ersetzt in einer Situation `chess_board` eine Dame in der obersten Reihe durch einen Bauern in Reihe 7.

In [ ]:
def replace_queen(chess_board):
    queen_square = chess_board.pieces(chess.QUEEN, chess.WHITE).pop()
    chess_board.remove_piece_at(queen_square)
    chess_board.set_piece_at(queen_square - 8, chess.Piece.from_symbol('P'))
    
    return chess_board

Die Funktion ``need_pawn_moves(chess_board)`` überprüft, ob Bauern-Züge für eine Situation `chess_board` beachtet werden müssen.
Hierfür verwendet sie das `g_user_wants_pawn` Flag, die Farbe am Zug, und ob sich Bauern in der Stellung oder eine Dame in der obersten Reihe befinden.

In [ ]:
def need_pawn_moves(chess_board):
    return g_user_wants_pawn \
           and chess_board.turn \
           and (check_for_pawn(chess_board) or check_top_row_for_queen(chess_board))

Die Funktion `moves_with_pawns(chess_board)` überprüft, ob eine Dame ersetzt oder ein Bauer gezogen werden muss, und führt den benötigten Schritt durch.

In [ ]:
def moves_with_pawns(chess_board):
    if check_for_pawn(chess_board):
        reached_boards = move_pawn(chess_board)
    else:
        reached_boards = [replace_queen(chess_board)]

    return reached_boards

## Hilfsfunktionen für die Rechnung

Die folgenden Funktionen werden bei der Berechnung von $S_n$-Mengen verwendet.

Die Funktion `keep_if_valid(chess_boards)` sortiert aus einer Liste `chess_boards` die nicht validen Stellungen aus und gibt die validen als `res` zurück.

In [ ]:
def keep_if_valid(chess_boards):
    res = []
    for board in chess_boards:
        if board.is_valid():
            res.append(board)    
    return res

Wenn mittels der KI eine Spielsituation ausgewertet wird, kann für jede Aktion des weißen Spielers ein Halbzug ausgewählt werden.
Für die Stellungen, bei denen Schwarz am Zug ist, muss die KI alle möglichen Züge auswerten können.
Da jedoch für einen spezifischen Zug, welcher eine Stellung von $S_n$ in $S_{n-1}$ führt, dasselbe nicht für alle Züge gilt, welche in der Stellung möglich sind, müssen die Stellungen, bei welchen Schwarz am Zug ist, besonders gefiltert werden.
Für jede Stellung $b$ aus einem $S_n$ mit $n = 2i$ muss folglich gelten:  
$$
b \in S_n \implies \forall m \in valid\_moves(b): b.push(m) \in S_{j} \land j < n
$$
Wobei `valid_moves` die Liste der legalen Züge für eine Stellung ist und `b.push(m)` die Stellung beschreibt, welche durch Ausführen des Zuges $m$ entsteht. 

Die Funktion `check_black_determinism(chess_boards)` stellt dies sicher.
Für jede Stellung in `chess_boards` (Teilmenge von $S_n$) wird jeder mögliche legale Halbzug ausgeführt und überprüft, ob die entstehende Stellung in einer Menge
$S_j$ mit $j < n$ auffindbar ist. Nur wenn alle Halbzüge diese Bedingung erfüllen, wird das Objekt in die Liste `deterministic` aufgenommen und zurückgegeben.

In [ ]:
def check_black_determinism(chess_boards):
    deterministic = []
    for chess_board in chess_boards:
        include = True
        for move in chess_board.legal_moves:
            chess_board.push(move)
            if len(chess_board.piece_map()) == len(g_piece_list):
                rep = to_integer(chess_board, g_piece_list)
            else:
                rep = -1
            chess_board.pop()
            if rep not in g_used_boards:
                include = False
                break
        
        if include:
            deterministic.append(chess_board)
            
    return deterministic

Nachdem die erste Menge $S_0$ berechnet wurde, müssen iterativ alle verbleibenden $n$ bestimmt werden.
Diese Aufgabe übernimmt die Funktion `previous_states(sn)`. Ausgehend von einer Menge an Integern `sn` wird die Menge $S_{n+1}$ (`sn_p_1`) bestimmt.

Hierfür wird über jede Stellung in der Menge `sn` iteriert, relevante Spielzüge ausgeführt (unterschieden zwischen Situationen mit Bauern und ohne), Duplikate entfernt und die erreichten Situationen der Menge $S_{n+1}$ hinzugefügt.
Die Situationen sowie alle Spiegelungen dieser werden ebenfalls der globalen Variable `g_used_boards` hinzugefügt.
Ist $n$ gerade ($n = 2i$), wird mit der Funktion `check_black_determinism(chess_boards)` weiter überprüft, ob alle Spielzüge in einer Menge $S_m$ mit $m < n$ enden.

In [ ]:
def previous_states(sn):
    sn_p_1 = set()
    for board_int in sn:
        chess_board = to_board(board_int, g_piece_list)
        chess_board.turn = chess_board.turn ^ True
        
        if need_pawn_moves(chess_board):
            reached_boards = moves_with_pawns(chess_board)
        else:
            reached_boards = moves_without_pawns(chess_board)
        
        reached_boards = keep_if_valid(reached_boards)    
        
        if not chess_board.turn:
            reached_boards = check_black_determinism(reached_boards)          
        
        reached_integers = multiple_to_integer(reached_boards, g_piece_list)
        
        # Remove unwanted mirrors
        reached_integers = reached_integers - g_used_boards
        
        sn_p_1 |= reached_integers
        add_all_to_used(reached_integers)

    return sn_p_1

### Funktionen für die Ergebnispersistierung

Damit die bestimmten Endspieltabellen später verwendet werden können, müssen sie abgespeichert werden. Zunächst wird die `g_piece_list` mit `pickle` in eine Zip-Datei geschrieben.
Während der Berechnung wird, damit die Anwendung weniger Arbeitsspeicher benötigt, jede vollständig bestimmte $S_n$ Menge als Pickle-File in dieselbe Zip-Datei geschrieben. Es werden hierbei nur die "ungespiegelten" Situationen gespeichert, da dies speichereffizienter ist. Spiegelungen können bei der Auswertung erneut bestimmt werden.

Beim Speichern der `g_piece_list` sowie der $S_n$ Mengen entstehen temporäre Daten.
Diese werden von der Funktion `delete_temp_file(filename)` gelöscht.
Die temporären Daten enden alle in `.pickle`, weswegen nur der Dateiname und nicht die Endung übergeben werden muss.

In [ ]:
def delete_temp_file(filename):
    if os.path.exists("S_n_Results/" + filename + ".pickle"):
            os.remove("S_n_Results/" + filename + ".pickle")

Die Funktion `store_piece_list()` wird immer als erste speichernde Funktion aufgerufen.
Sie erstellt eine Zip-Datei mit der Dateiendung `.chessAI` und speichert die gepickelte `g_piece_list` in dieser.
Da die Funktion eine neue Datei erstellt, muss der Zähler für die Anzahl der gespeicherten $S_n$ Mengen zurückgesetzt werden.

In [ ]:
g_stored_set_counter = 0

def store_piece_list():
    global g_stored_set_counter
    g_stored_set_counter = 0
    filename = "piece_list"
    with open("S_n_Results/" + filename + ".pickle", "wb") as pickleFile:
        pickleFile.write(pickle.dumps(g_piece_list))
        pickleFile.close()
        
    with ZipFile("S_n_Results/" + g_filename + '.chessAI', 'w', compression=ZIP_DEFLATED) as zipped:
        zipped.write("S_n_Results/" + filename + ".pickle", filename + ".pickle")
        zipped.close()
        
    delete_temp_file(filename)

Die Funktion `store_sn(sn)` fügt die Inhalte der Menge `sn` als `pickle` Datei in die `.chessAI` Datei ein.
Die temporäre `.pickle` Datei wird anschließend gelöscht.
Der Dateiname der Menge hängt vom globalen Zähler der gespeicherten Mengen ab.

In [ ]:
def store_sn(sn):
    global g_stored_set_counter
    filename = "S_" + str(g_stored_set_counter)
    
    with open("S_n_Results/" + filename + ".pickle", "wb") as pickleFile:
        pickleFile.write(pickle.dumps(sn))
        pickleFile.close()
    
    with ZipFile("S_n_Results/" + g_filename + '.chessAI', 'a', compression=ZIP_DEFLATED) as zipped:
        zipped.write("S_n_Results/" + filename + ".pickle", filename + ".pickle")
        zipped.close()
    
    g_stored_set_counter += 1
    
    delete_temp_file(filename)

### Orchestrierung der zuvor definierten Funktionen zum Berechnen der Endspieltabellen


Die Funktion `calculate()` kann aufgerufen werden, um mit den oben angegebenen Einstellungen eine neue Berechnung durchzuführen. 

Die Funktion berechnet zuerst die Ausgangssituation $S_0$ und speichert sie.
Anschließend werden weitere $n$ in einer Schleife bestimmt.
Nach jeder Berechnung eines $n$, werden die bestimmten Spielsituationen in der `.chessAI` gespeichert.
Die Funktionen, welche für die Berechnung nötig sind, wurden im Verlauf dieses Notebooks aufgeführt und erklärt.

Wenn der Nutzer einen Bauern in die `g_piece_list` eingetragen hat, wird dieser durch eine Dame ersetzt.
Dies passiert, da die Dame der mächtigste Tausch ist, und alle gewonnenen Situationen für $S_0$ diese Dame enthalten müssen. Durch das `g_user_wants_pawn` Flag wird sichergestellt, dass der Bauer in den Tabellen vorkommt.  


In [ ]:
def calculate():
    global g_piece_list
    if g_user_wants_pawn:
        g_piece_list[g_piece_list.index(chess.Piece.from_symbol('P'))] = chess.Piece.from_symbol('Q')
    store_piece_list()
    
    count = 0
    sn = calculate_s0()
    print("Done with S" + str(count))
    store_sn(sn)  
    
    while len(sn) != 0:
        sn = previous_states(sn)
        store_sn(sn)
        count += 1
        print("Done with S" + str(count))

Die Funktion `run()` aggregiert die zur Berechnung und Speicherung benötigten Funktionen und führt sie in benötigter Reihenfolge aus.
Zusätzlich werden die globalen Variablen `g_used_boards` und `g_user_wants_pawn` zurückgesetzt.

In [ ]:
def run():
    global g_used_boards
    global g_user_wants_pawn
    g_used_boards = set()
    g_user_wants_pawn = chess.Piece.from_symbol("P") in g_piece_list
    calculate()
    print("Done")
